In [1]:
import os
import json
import ast
import numpy as np
import pandas as pd
from collections import Counter
pd.set_option('display.max_colwidth', 900)
from sklearn import linear_model

In [2]:
def parse_draft_to_flooey_format(draft):
    players = draft['picks']
    cards_per_pack = int(len(players[0]) / 3)
    packs = []
    for pack_index in range(3):
        choices = []
        for pick_index in range(cards_per_pack):
            options = []
            idx = pick_index
            while idx < cards_per_pack:
                options.append(players[(idx - pick_index) % 8][idx + pack_index * cards_per_pack])
                idx += 1
            choices.append(options)
        packs.append(choices)
    return packs

In [10]:
def get_cardset(drafts):
    cardset = set()
    for draft in drafts:
        for pack in draft['picks']:
            for card in pack:
                cardset.add(card)
    carddict = {card: i for i, card in enumerate(cardset)}
    carddict['<unk>'] = -1
    return carddict

In [15]:
def vectorize_drafts(drafts):
    cardset = get_cardset(drafts)
    flooey_drafts = [parse_draft_to_flooey_format(draft) for draft in drafts]
    return [vectorize_draft(draft, cardset) for draft in flooey_drafts]

def vectorize_draft(draft, cardset):
    draft_x = np.zeros([3, len(draft[0][0]), len(cardset) * 2])
    draft_y = np.zeros([3, len(draft[0][0]), len(cardset)])
    pile = np.zeros(len(cardset))
    for pa, pack in enumerate(draft):
        for pi, pick in enumerate(pack):
            draft_y[pa][pi][cardset.get(pick[0], -1)] = 1
            x1 = np.zeros(len(cardset))
            for card in pick:
                if card in cardset:
                    x1[cardset[card]] += 1
                else:
                    print("Unknown card:", card)
                    x1[-1] += 1
            draft_x[pa][pi] = np.append(x1, pile)
            pile[cardset.get(pick[0], -1)] += 1
            
    return {'draft_x': draft_x, 'draft_y': draft_y}

In [5]:
def draft_to_df(draftx, drafty):
    picks = []
    piles = []
    selections = []
    for (dx, dy) in zip(draftx, drafty):
        pick = []
        pile = []
        for card in np.where(dx != 0)[0]:
            if card < len(cardset):
                for i in range(int(dx[card])):
                    pick.append(invcardset[card])
            else:
                for i in range(int(dx[card])):
                    pile.append(invcardset[card - len(cardset)])
        picks.append(pick)
        piles.append(pile)
        selections.append(invcardset[np.where(dy==1)[0][0]])
    return pd.DataFrame({'picks': picks, 'piles': piles, 'selections': selections})

In [41]:
reg = linear_model.SGDClassifier()

In [42]:
mtgset = 'DOM'
alldrafts = os.listdir('drafts/' + mtgset)
drafts = [ast.literal_eval(open('drafts/' + mtgset + '/' + draft, 'r').read()) for draft in alldrafts[:10000]]
cardset = get_cardset(drafts)
invcardset = {v: k for k, v in cardset.items()}

In [ ]:
batch = 10000

for j in range(0, len(alldrafts) - batch, batch):
    drafts = []
    for i, draft in enumerate(alldrafts[j:j + batch]):
        drafts.append(ast.literal_eval(open('drafts/' + mtgset + '/' + draft, 'r').read()))

    flooey_drafts = [parse_draft_to_flooey_format(draft) for draft in drafts]
    draftsv = []
    for i, draft in enumerate(flooey_drafts):
        draftsv.append(vectorize_draft(draft, cardset))
    X, Y = [], []
    for draft in draftsv:
        draft_x = draft['draft_x']
        draft_y = draft['draft_y']
        for pack in range(len(draft_x)):
            for pick in range(len(draft_x[pack])):
                Y.append(draft_y[pack][pick])
                X.append(draft_x[pack][pick].flatten())
    
    if j == 0:
        print('Test set made.')
        x_test = X
        y_test = Y
        continue
    reg.partial_fit(X, np.argmax(Y, axis=1), classes=range(len(Y[0])))
    ypred = reg.predict(x_test[:])
    ytrue = [np.argmax(t) for t in y_test]
    xpickdf = []
    xpiledf = []
    for pick in x_test:
        xpile = Counter()
        xpick = []
        for card in np.where(phttp://localhost:8888/notebooks/DraftBot-SimpleLinear.ipynb#ick != 0)[0]:
            if card < len(cardset):
                for k in range(int(pick[card])):
                    xpick.append(invcardset[card])
            else:
                xpile[invcardset[card - len(cardset)]] += 1
        xpickdf.append(xpick)
        xpiledf.append(xpile)
    df = pd.DataFrame({'xpick': xpickdf, 'xpile': xpiledf, 'ypred': ypred, 'ytrue': ytrue})
    print("Accuracy after %s examples: %s" % (j, len(df[df.ypred == df.ytrue]) * 1.0 / len(df)))

In [51]:
df['Predicted Card'] = df['ypred'].apply(lambda x: invcardset[x])
df['Selected Card'] = df['ytrue'].apply(lambda x: invcardset[x])
print("Accuracy: %s", len(df[df.ypred == df.ytrue]) * 1.0 / len(df))

('Accuracy: %s', 0.4958690476190476)


In [53]:
# Sample Draft
df.iloc[42*99:42*100][['xpick', 'xpile', 'Predicted Card', 'Selected Card']]

,xpick,xpile,Predicted Card,Selected Card
4158,"[Memorial_to_Genius, Garna_the_Bloodflame, Steel_Leaf_Champion, Blessed_Light, Arcane_Flight, Fiery_Intervention, Sanctum_Spirit, Adventurous_Impulse, Ghitu_Lavarunner, Thallid_Omnivore, Blink_of_an_Eye, Charge, Relic_Runner, Seismic_Shift]",{},Steel_Leaf_Champion,Steel_Leaf_Champion
4159,"[Frenzied_Rage, Tiana_Ship's_Caretaker, Divination, Arcane_Flight, Unwind, Dub, Stronghold_Confessor, Saproling_Migration, Charge, Ghitu_Chronicler, Untamed_Kavu, Run_Amok, Goblin_Warchief]",{u'Steel_Leaf_Champion': 1},Untamed_Kavu,Untamed_Kavu
4160,"[Shanna_Sisay's_Legacy, Divination, Soul_Salvage, Warlord's_Fury, D'Avenant_Trapper, Mammoth_Spider, Saproling_Migration, Benalish_Honor_Guard, Gift_of_Growth, Mox_Amber, Skirk_Prospector, Run_Amok]","{u'Steel_Leaf_Champion': 1, u'Untamed_Kavu': 1}",Saproling_Migration,Saproling_Migration
4161,"[Excavation_Elephant, Memorial_to_Folly, Lingering_Phantom, Rescue, Caligo_Skin-Witch, Sergeant-at-Arms, Homarid_Explorer, Keldon_Overseer, Mammoth_Spider, The_Mirari_Conjecture, Powerstone_Shard]","{u'Steel_Leaf_Champion': 1, u'Untamed_Kavu': 1, u'Saproling_Migration': 1}",Mammoth_Spider,Mammoth_Spider
4162,"[Pegasus_Courser, Opt, Rescue, Soul_Salvage, Keldon_Warcaller, Sulfur_Falls, Academy_Drake, Ghitu_Journeymage, Krosan_Druid, Knight_of_New_Benalia]","{u'Steel_Leaf_Champion': 1, u'Untamed_Kavu': 1, u'Saproling_Migration': 1, u'Mammoth_Spider': 1}",Soul_Salvage,Krosan_Druid
4163,"[Skittering_Surveyor, Bloodstone_Goblin, Blessed_Light, Feral_Abomination, Unwind, Syncopate, Keldon_Raider, Arbor_Armament, Rat_Colony]","{u'Steel_Leaf_Champion': 1, u'Untamed_Kavu': 1, u'Saproling_Migration': 1, u'Krosan_Druid': 1, u'Mammoth_Spider': 1}",Blessed_Light,Arbor_Armament
4164,"[Excavation_Elephant, Frenzied_Rage, Opt, Artificer's_Assistant, Memorial_to_War, Call_the_Cavalry, Ghitu_Lavarunner, Fervent_Strike]","{u'Arbor_Armament': 1, u'Mammoth_Spider': 1, u'Untamed_Kavu': 1, u'Krosan_Druid': 1, u'Steel_Leaf_Champion': 1, u'Saproling_Migration': 1}",Call_the_Cavalry,Frenzied_Rage
4165,"[Howling_Golem, Healing_Grace, Rescue, Feral_Abomination, Warlord's_Fury, Academy_Drake, Ghitu_Chronicler]","{u'Arbor_Armament': 1, u'Mammoth_Spider': 1, u'Untamed_Kavu': 1, u'Frenzied_Rage': 1, u'Steel_Leaf_Champion': 1, u'Krosan_Druid': 1, u'Saproling_Migration': 1}",Academy_Drake,Warlord's_Fury
4166,"[Arcane_Flight, Sanctum_Spirit, Ghitu_Lavarunner, Charge, Relic_Runner, Seismic_Shift]","{u'Frenzied_Rage': 1, u'Arbor_Armament': 1, u'Mammoth_Spider': 1, u'Untamed_Kavu': 1, u'Warlord's_Fury': 1, u'Steel_Leaf_Champion': 1, u'Krosan_Druid': 1, u'Saproling_Migration': 1}",Ghitu_Lavarunner,Ghitu_Lavarunner
4167,"[Frenzied_Rage, Tiana_Ship's_Caretaker, Divination, Unwind, Charge]","{u'Frenzied_Rage': 1, u'Krosan_Druid': 1, u'Arbor_Armament': 1, u'Untamed_Kavu': 1, u'Warlord's_Fury': 1, u'Mammoth_Spider': 1, u'Steel_Leaf_Champion': 1, u'Saproling_Migration': 1, u'Ghitu_Lavarunner': 1}",Divination,Frenzied_Rage


In [ ]:
# Scratch work, ignore

In [223]:
mtgset = 'DOM'
reg = linear_model.SGDRegressor()
drafts = []
for i, draft in enumerate(os.listdir('drafts/' + mtgset)[:30000]):
    drafts.append(ast.literal_eval(open('drafts/' + mtgset + '/' + draft, 'r').read()))
#     if i % 10000 == 1:
#         print(i,)

cardset = get_cardset(drafts)
invcardset = {v: k for k, v in cardset.items()}
flooey_drafts = [parse_draft_to_flooey_format(draft) for draft in drafts]

In [224]:
draftsv = []
for i, draft in enumerate(flooey_drafts[:30000]):
#     if i % 5000 == 1:
#         print(i,)
    draftsv.append(vectorize_draft(draft, cardset))

In [225]:
X, Y = [], []
for draft in draftsv:
    draft_x = draft['draft_x']
    draft_y = draft['draft_y']
    for pack in range(len(draft_x)):
        for pick in range(len(draft_x[pack])):
            Y.append(draft_y[pack][pick])
            X.append(draft_x[pack][pick].flatten())
# X = [draft['draft_x'] for draft in draftsv]
# Y = [draft['draft_y'] for draft in draftsv]

In [228]:
draft_to_df(X[:42], Y[:42])

,picks,piles,selections
0,"[Baird_Steward_of_Argive, Cloudreader_Sphinx, Cabal_Evangel, The_Eldest_Reborn, Navigator's_Compass, Sparring_Construct, Llanowar_Envoy, Hinterland_Harbor, Stronghold_Confessor, Settle_the_Score, Ghitu_Lavarunner, Thallid_Omnivore, Arbor_Armament, Serra_Disciple]",[],The_Eldest_Reborn
1,"[Broken_Bond, Slinn_Voda_the_Rising_Deep, Sylvan_Awakening, Artificer's_Assistant, Weight_of_Memory, Arcane_Flight, Demonic_Vigor, Llanowar_Envoy, Unwind, Call_the_Cavalry, Fervent_Strike, Arbor_Armament, Run_Amok]",[The_Eldest_Reborn],Sylvan_Awakening
2,"[Baloth_Gorger, Memorial_to_Folly, Primordial_Wurm, Hinterland_Harbor, Tolarian_Scholar, Cold-Water_Snapper, Stronghold_Confessor, On_Serra's_Wings, Keldon_Overseer, Aven_Sentry, Relic_Runner, Guardians_of_Koilos]","[Sylvan_Awakening, The_Eldest_Reborn]",Baloth_Gorger
3,"[Ancient_Animus, Gideon's_Reproach, Radiating_Lightning, Shivan_Fire, Befuddle, Ghitu_Lavarunner, Urgoros_the_Empty_One, Amaranthine_Wall, Guardians_of_Koilos, Arbor_Armament, Naban_Dean_of_Iteration]","[Baloth_Gorger, Sylvan_Awakening, The_Eldest_Reborn]",Urgoros_the_Empty_One
4,"[Artificer's_Assistant, Adamant_Will, Caligo_Skin-Witch, Call_the_Cavalry, Cabal_Paladin, Homarid_Explorer, Fire_Elemental, Wizard's_Retort, Powerstone_Shard, Rat_Colony]","[Baloth_Gorger, Sylvan_Awakening, The_Eldest_Reborn, Urgoros_the_Empty_One]",Cabal_Paladin
5,"[Windgrace_Acolyte, Drudge_Sentinel, Rona_Disciple_of_Gix, Unwind, Tragic_Poet, Mammoth_Spider, Powerstone_Shard, Rat_Colony, Run_Amok]","[Baloth_Gorger, Sylvan_Awakening, The_Eldest_Reborn, Cabal_Paladin, Urgoros_the_Empty_One]",Windgrace_Acolyte
6,"[Rona_Disciple_of_Gix, Pardic_Wanderer, Keldon_Warcaller, Mesa_Unicorn, Dark_Bargain, Gift_of_Growth, Rat_Colony, Serra_Disciple]","[Baloth_Gorger, Windgrace_Acolyte, Sylvan_Awakening, The_Eldest_Reborn, Cabal_Paladin, Urgoros_the_Empty_One]",Dark_Bargain
7,"[Pierce_the_Sky, Short_Sword, Aesthir_Glider, Blessing_of_Belzenlok, Memorial_to_Glory, Guardians_of_Koilos, Invoke_the_Divine]","[Baloth_Gorger, Windgrace_Acolyte, Sylvan_Awakening, The_Eldest_Reborn, Cabal_Paladin, Urgoros_the_Empty_One, Dark_Bargain]",Blessing_of_Belzenlok
8,"[Cloudreader_Sphinx, Cabal_Evangel, Stronghold_Confessor, Ghitu_Lavarunner, Arbor_Armament, Serra_Disciple]","[Baloth_Gorger, Windgrace_Acolyte, Sylvan_Awakening, The_Eldest_Reborn, Blessing_of_Belzenlok, Cabal_Paladin, Urgoros_the_Empty_One, Dark_Bargain]",Stronghold_Confessor
9,"[Slinn_Voda_the_Rising_Deep, Artificer's_Assistant, Arcane_Flight, Unwind, Arbor_Armament]","[Baloth_Gorger, Windgrace_Acolyte, Sylvan_Awakening, The_Eldest_Reborn, Stronghold_Confessor, Blessing_of_Belzenlok, Cabal_Paladin, Urgoros_the_Empty_One, Dark_Bargain]",Arcane_Flight


In [226]:
s = 1200000
x_train = X[:s]
y_train = Y[:s]
x_test = X[s:]
y_test = Y[s:]

In [202]:
reg.partial_fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [203]:
# reg.score(x_test, y_test)

In [211]:
y_pred = reg.predict(x_test[:])

In [212]:
ypred = [np.argmax(t) for t in y_pred]
ytrue = [np.argmax(t) for t in y_test]
xpickdf = []
xpiledf = []
for pick in x_test:
    xpile = Counter()
    xpick = []
    for card in np.where(pick == 1)[0]:
        if card < len(cardset):
            xpick.append(invcardset[card])
        else:
            xpile[invcardset[card - len(cardset)]] += 1
    xpickdf.append(xpick)
    xpiledf.append(xpile)

In [213]:
df = pd.DataFrame({'xpick': xpickdf, 'xpile': xpiledf, 'ypred': ypred, 'ytrue': ytrue})

In [217]:
len(df[df.ypred == df.ytrue]) * 1.0 / len(df)

0.4054833333333333

In [218]:
df['ypred_card'] = df['ypred'].apply(lambda x: invcardset[x])
df['ytrue_card'] = df['ytrue'].apply(lambda x: invcardset[x])

In [220]:
i = 150
df.iloc[i:i + 42]

,xpick,xpile,ypred,ytrue,ypred_card,ytrue_card
150,"[Excavation_Elephant, Skittering_Surveyor, Cabal_Evangel, The_Flame_of_Keld, Shivan_Fire, Oath_of_Teferi, Befuddle, Bloodtallow_Candle, Keldon_Warcaller, Tatyova_Benthic_Druid, Wild_Onslaught, Dark_Bargain, Krosan_Druid, Arbor_Armament]",{},146,146,Tatyova_Benthic_Druid,Tatyova_Benthic_Druid
151,"[Broken_Bond, Garna_the_Bloodflame, Fungal_Plots, Cabal_Evangel, Academy_Journeymage, Deathbloom_Thallid, Unwind, Cold-Water_Snapper, Keldon_Warcaller, Aven_Sentry, Shield_of_the_Realm, Grow_from_the_Ashes, Run_Amok]",{u'Tatyova_Benthic_Druid': 1},75,228,Academy_Journeymage,Grow_from_the_Ashes
152,"[Frenzied_Rage, Adeliz_the_Cinder_Wind, Rescue, Academy_Journeymage, Memorial_to_War, Demonic_Vigor, Pardic_Wanderer, Keldon_Warcaller, Sorcerer's_Wand, Gift_of_Growth, Arbor_Armament, Serra_Disciple]","{u'Grow_from_the_Ashes': 1, u'Tatyova_Benthic_Druid': 1}",62,75,Adeliz_the_Cinder_Wind,Academy_Journeymage
153,"[Song_of_Freyalise, Blessed_Light, Divest, Homarid_Explorer, Fire_Elemental, Tetsuko_Umezawa_Fugitive, Dark_Bargain, Krosan_Druid, Grow_from_the_Ashes, Ghitu_Chronicler, Skirk_Prospector]","{u'Grow_from_the_Ashes': 1, u'Tatyova_Benthic_Druid': 1, u'Academy_Journeymage': 1}",35,35,Song_of_Freyalise,Song_of_Freyalise
154,"[Windgrace_Acolyte, Gaea's_Blessing, Keldon_Warcaller, Tragic_Poet, Charge, Dark_Bargain, Krosan_Druid, Thran_Temporal_Gateway, Powerstone_Shard, Gift_of_Growth]","{u'Grow_from_the_Ashes': 1, u'Song_of_Freyalise': 1, u'Tatyova_Benthic_Druid': 1, u'Academy_Journeymage': 1}",205,225,Thran_Temporal_Gateway,Gift_of_Growth
155,"[Zhalfirin_Void, Frenzied_Rage, Arcane_Flight, Divest, Pardic_Wanderer, Ghitu_Lavarunner, Thallid_Omnivore, Wizard's_Retort, Gaea's_Protector]","{u'Grow_from_the_Ashes': 1, u'Gift_of_Growth': 1, u'Song_of_Freyalise': 1, u'Tatyova_Benthic_Druid': 1, u'Academy_Journeymage': 1}",203,203,Wizard's_Retort,Wizard's_Retort
156,"[Skittering_Surveyor, Jaya_Ballard, Cabal_Evangel, Adamant_Will, Nature's_Spiral, Gaea's_Protector, Gift_of_Growth, Whisper_Blood_Liturgist]","{u'Gift_of_Growth': 1, u'Wizard's_Retort': 1, u'Academy_Journeymage': 1, u'Song_of_Freyalise': 1, u'Tatyova_Benthic_Druid': 1, u'Grow_from_the_Ashes': 1}",56,163,Jaya_Ballard,Nature's_Spiral
157,"[Slinn_Voda_the_Rising_Deep, Short_Sword, Lingering_Phantom, Llanowar_Envoy, Adamant_Will, Cabal_Paladin, Skirk_Prospector]","{u'Gift_of_Growth': 1, u'Wizard's_Retort': 1, u'Academy_Journeymage': 1, u'Song_of_Freyalise': 1, u'Tatyova_Benthic_Druid': 1, u'Nature's_Spiral': 1, u'Grow_from_the_Ashes': 1}",37,106,Short_Sword,Llanowar_Envoy
158,"[Cabal_Evangel, Oath_of_Teferi, Keldon_Warcaller, Dark_Bargain, Krosan_Druid, Arbor_Armament]","{u'Llanowar_Envoy': 1, u'Gift_of_Growth': 1, u'Wizard's_Retort': 1, u'Academy_Journeymage': 1, u'Song_of_Freyalise': 1, u'Tatyova_Benthic_Druid': 1, u'Nature's_Spiral': 1, u'Grow_from_the_Ashes': 1}",78,204,Oath_of_Teferi,Krosan_Druid
159,"[Broken_Bond, Cabal_Evangel, Unwind, Cold-Water_Snapper, Keldon_Warcaller]","{u'Llanowar_Envoy': 1, u'Gift_of_Growth': 1, u'Wizard's_Retort': 1, u'Academy_Journeymage': 1, u'Song_of_Freyalise': 1, u'Tatyova_Benthic_Druid': 1, u'Nature's_Spiral': 1, u'Grow_from_the_Ashes': 1, u'Krosan_Druid': 1}",107,18,Unwind,Broken_Bond


In [133]:
len(df[df.ypred == df.ytrue])

116272

In [134]:
len(df)

250000